In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf


# Download the SPY data from Yahoo Finance and preprocess it
spy = yf.Ticker("SPY")
data = spy.history(period="max")
ts = data['Close']
ts = ts.resample('D').ffill() # resample to daily frequency and forward fill missing values


# Define the transition model
def random_walk(particles, t, std_noise):
    return particles + np.random.normal(loc=0, scale=std_noise, size=particles.shape)

# Define the observation model
def gaussian_likelihood(obs, particles):
    return np.exp(-0.5 * np.sum((obs - particles)**2, axis=1))

# Define the resampling strategy
def multinomial_resampling(weights):
    return np.random.choice(len(weights), size=len(weights), replace=True, p=weights)

# Define the particle filter function
def particle_filter(data, n_particles, n_steps, std_noise, transition_model, observation_model, resampling_strategy):
    n_timesteps = len(data)
    if data.ndim == 1:
        n_features = 1
    else:
        n_features = data.shape[1]
    particles = np.zeros((n_particles, n_features))
    weights = np.ones(n_particles) / n_particles
    resampled = False
    forecasts_mean = np.zeros(n_steps)
    forecasts_std = np.zeros(n_steps)
    for t in range(n_timesteps):
        # Resampling: resample particles according to their weights
        if resampling_strategy == 'multinomial':
            if np.sum(weights ** 2) < 1e-16:
                particles = np.random.normal(loc=particles.mean(axis=0), scale=std_noise, size=particles.shape)
                weights = np.ones(n_particles) / n_particles
                resampled = True
            else:
                indexes = np.random.choice(n_particles, n_particles, p=weights)
                particles = particles[indexes]
                weights = np.ones(n_particles) / n_particles
                resampled = False
        elif resampling_strategy == 'systematic':
            raise NotImplementedError('Systematic resampling not implemented yet')
        # Transition: sample new particles from the transition model
        particles = transition_model(particles, t, std_noise)
        # Compute the average likelihood over the n_lags observations
        likelihoods = observation_model(data[t], particles)
        likelihoods += 1e-300  # add a small constant to avoid zero/nan values
        weights *= likelihoods
        weights /= np.sum(weights)
        # Resampling: resample particles according to their weights
        if resampling_strategy == 'multinomial' and not resampled:
            if np.sum(weights ** 2) < 1e-16:
                particles = np.random.normal(loc=particles.mean(axis=0), scale=std_noise, size=particles.shape)
                weights = np.ones(n_particles) / n_particles
            else:
                indexes = np.random.choice(n_particles, n_particles, p=weights)
                particles = particles[indexes]
                weights = np.ones(n_particles) / n_particles
        elif resampling_strategy == 'systematic':
            raise NotImplementedError('Systematic resampling not implemented yet')
        # Compute the forecast for the next n_steps timesteps
        if t == n_timesteps - 1:
            for i in range(n_steps):
                forecast_mean = np.sum(weights * particles.T[0])
                forecast_std = np.sqrt(np.sum(weights * (particles.T[0] - forecast_mean) ** 2))
                forecasts_mean[i] = forecast_mean
                forecasts_std[i] = forecast_std
        else:
            continue
    return forecasts_mean, forecasts_std

# Set the parameters
n_particles = 10000
n_steps = 10
std_noise = 0.05

# Run the particle filter algorithm on the SPY data
forecasts_mean, forecasts_std = particle_filter(ts.values, n_particles, n_steps, std_noise, random_walk, gaussian_likelihood, multinomial_resampling)




In [2]:
forecasts_mean

array([9.56961425, 9.56961425, 9.56961425, 9.56961425, 9.56961425,
       9.56961425, 9.56961425, 9.56961425, 9.56961425, 9.56961425])

In [3]:
forecasts_std

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])